LAB - 4
13.09.23


In [8]:
import numpy as np

# Define your training data
training_data = [
    ("Put, coat, back, Chair, sat, down", "Furniture"),
    ("Chair, made, timber, company", "Furniture"),
    ("Chair, institute, best", "Position"),
    ("Award, IT, Chair", "Position"),
    ("Type, different, Chair, Award, fun, use", "Furniture"),
    ("Award, Chair, IT, company", "Furniture"),
]

# Function to train the Bayesian model
def train_bayesian_model(training_data):
    category_counts = {}
    keyword_counts = {}

    # Initialize counts
    for sentence, category in training_data:
        if category not in category_counts:
            category_counts[category] = 0
        category_counts[category] += 1

        keywords = sentence.split(", ")
        for i in keywords:
            if i not in keyword_counts:
                keyword_counts[i] = {}
            if category not in keyword_counts[i]:
                keyword_counts[i][category] = 0
            keyword_counts[i][category] += 1

    # Calculate prior probabilities
    total_sentences = len(training_data)
    prior_probabilities = {category: count / total_sentences for category, count in category_counts.items()}
    
    # Calculate likelihoods
    likelihoods = {}
    for keyword, category_counts in keyword_counts.items():
        likelihoods[keyword] = {category: count / category_counts[category] for category, count in category_counts.items()}

    return prior_probabilities, likelihoods

# Function to disambiguate a sentence
def disambiguate_sentence(sentence, prior_probabilities, likelihoods):
    keywords = sentence.split(", ")
    category_probabilities = {}

    for category, prior in prior_probabilities.items():
        likelihood_product = 1.0

        for keyword in keywords:
            if keyword in likelihoods and category in likelihoods[keyword]:
                likelihood_product *= likelihoods[keyword][category]
            else:
                likelihood_product *= 0.5  # A default value for missing likelihoods

        category_probabilities[category] = prior * likelihood_product

    # Normalize probabilities
    total_probability = sum(category_probabilities.values())
    for category in category_probabilities:
        category_probabilities[category] /= total_probability

    return category_probabilities

# Train the Bayesian model
prior_probabilities, likelihoods = train_bayesian_model(training_data)

# Given sentence for disambiguation
sentence = "Award, Chair, IT, company"

# Disambiguate the sentence
result = disambiguate_sentence(sentence, prior_probabilities, likelihoods)

# Print the results
for category, probability in result.items():
    print(f"Posterior Probability of {category}: {probability:.2f}")

Posterior Probability of Furniture: 0.80
Posterior Probability of Position: 0.20


In [40]:
import csv
import numpy as np

# Function to train the Bayesian model
def train_bayesian_model(training_data):
    category_counts = {}
    keyword_counts = {}
 
    # Initialize counts
    for sentence, category in training_data:
        if category not in category_counts:
            category_counts[category] = 0
        category_counts[category] += 1

        keywords = sentence.split(", ")
        for keyword in keywords:
            if keyword not in keyword_counts:
                keyword_counts[keyword] = {}
            if category not in keyword_counts[keyword]:
                keyword_counts[keyword][category] = 0
            keyword_counts[keyword][category] += 1

    # Calculate prior log probabilities
    total_sentences = len(training_data)
    prior_log_probabilities = {category: count / total_sentences for category, count in category_counts.items()}

    # Calculate log likelihoods
    log_likelihoods = {}
    for keyword, category_counts in keyword_counts.items():
        log_likelihoods[keyword] = {category: np.log10(count / category_counts[category]) for category, count in category_counts.items()}

    return prior_log_probabilities, log_likelihoods

# Function to disambiguate a sentence
def disambiguate_sentence(sentence, prior_log_probabilities, log_likelihoods):
    keywords = sentence.split(", ")
    category_log_probabilities = {}

    for category, prior_log_prob in prior_log_probabilities.items():
        log_posterior_probability = prior_log_prob

        for keyword in keywords:
            if keyword in log_likelihoods and category in log_likelihoods[keyword]:
                log_posterior_probability += log_likelihoods[keyword][category]
            else:
                # Use a default value for missing likelihoods
                log_posterior_probability += np.log10(0.5)

        category_log_probabilities[category] = log_posterior_probability

    # Normalize log probabilities
    max_log_prob = max(category_log_probabilities.values())
    category_probabilities = {category: 10 ** (log_prob - max_log_prob) for category, log_prob in category_log_probabilities.items()}

    return category_probabilities

# Load training data from a CSV file
def load_training_data_from_csv(csv_file):
    training_data = []
    with open(csv_file, mode='r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header row if it exists
        for row in reader:
            if len(row) == 2:
                sentence, category = row
                training_data.append((sentence, category))
    return training_data

# Define the CSV file name with your training data
csv_file = "training_data.csv"

# Load training data from the CSV file
training_data = load_training_data_from_csv(csv_file)

# Train the Bayesian model
prior_log_probabilities, log_likelihoods = train_bayesian_model(training_data)

# Given sentence for disambiguation
sentence = "Award, IT, company"

# Disambiguate the sentence
result = disambiguate_sentence(sentence, prior_log_probabilities, log_likelihoods)

# Print the results
for category, probability in result.items():
    print(f"Posterior Probability of {category}: {1-(100*probability:.6f)}")

SyntaxError: f-string: closing parenthesis '}' does not match opening parenthesis '(' (<ipython-input-40-e1c529c9a103>, line 86)